In [ ]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:
declare or replace qry_str string;

set var qry_str = 
  "create or replace view metv_dbsql
  (
    date comment 'Date when Databricks received the request.',
    client_application comment 'Client application that ran the statement. For example: Databricks SQL, Tableau, and Power BI.   Nulls and blanks are labled as Unknown',
    query_count comment 'Count of queries.',
    query_duration_sec_avg comment 'Sum of total execution time of the statement in seconds (Excluding result fetch time).',
    query_duration_sec_min comment 'Minimum of total execution time of the statement in seconds (Excluding result fetch time).',
    query_duration_sec_max comment 'Maximum of total execution time of the statement in seconds (Excluding result fetch time).',
    waiting_at_capacity_duration_sec_sum comment 'Sum of time spent waiting in queue for available compute capacity in seconds.',
    distinct_users comment 'Distinct count of users based on executed_by_user_id.',
    distinct_users_t7d comment 'Distinct count t7d of users based on executed_by_user_id.',
    distinct_users_t30d comment 'Distinct count t30d of users based on executed_by_user_id.'
  )
  with metrics
  language yaml
  as $$
  version: 0.1
  
  source: " || :catalog || '.' || :schema || ".vw_query_history
  
  joins:
    - name: calendar
      source: " || :catalog || '.' || :schema || ".vw_dim_calendar
      on: calendar.date_int = source.date_int
  
  dimensions:
    - name: date
      expr: calendar.calendar_date
    - name: client_application
      expr: source.client_application
  
  measures:
    - name: query_count
      expr: count(statement_id)
    - name: query_duration_sec_avg
      expr: avg(total_duration_ms / 1000)
    - name: query_duration_sec_min
      expr: min(total_duration_ms / 1000)
    - name: query_duration_sec_max
      expr: max(total_duration_ms / 1000)
    - name: waiting_at_capacity_duration_sec_sum
      expr: sum(waiting_at_capacity_duration_ms / 1000)
    - name: distinct_users
      expr: count(distinct executed_by_user_id)
    - name: distinct_users_t7d
      expr: distinct_users
      window:
        - order: date
          range: trailing 7 day
          semiadditive: last
    - name: distinct_users_t30d
      expr: distinct_users
      window:
        - order: date
          range: trailing 30 day
          semiadditive: last
  $$";

execute immediate qry_str;